<h2><b>This notebook and method are Not used in main production</b></h2>

<h4><b>This notebook attempts to extract event on the linguistic notations </b></h4>

In [2]:
import pandas as pd

In [3]:
from pandas import DataFrame

df1 = pd.read_csv(r'E:\Semesters\Fyp prepation\Deep_learning_prep/testing_Dataset_1.csv')
df1



,Headline,Name,Data_Time,URL,Artical
0,"-- Inco's Net Soars on Higher Metal Prices, Br...",-- Dale Crofts,-- 2006-10-20T20:16:16Z,-- http://www.bloomberg.com/news/2006-10-20/in...,"Inco Ltd., ..."
1,"-- Jim Cramer: Diageo, Anheuser-Busch, Monster...",-- Steven Bodzin,-- 2006-10-21T00:08:44Z,-- http://www.bloomberg.com/news/2006-10-21/ji...,Jim Cramer ...
2,-- EU Energy Chief Backs German Plan for Price...,-- Thomas Bauer,-- 2006-10-23T11:51:36Z,-- http://www.bloomberg.com/news/2006-10-23/eu...,European Un...
3,-- Ex-Plant Worker Shuster Pleads Guilty in Tr...,-- David Glovin,-- 2006-10-23T20:00:29Z,-- http://www.bloomberg.com/news/2006-10-23/ex...,A former wo...
4,"-- Jim Cramer: Bare Escentuals, Allergan, Medi...",-- Steven Bodzin,-- 2006-10-24T01:32:04Z,-- http://www.bloomberg.com/news/2006-10-24/ji...,Bare Escent...
...,...,...,...,...,...
448390,-- Rubber Drops to Two-Week Low After Forecast...,-- B y A y a T a k a d a,-- 2013-11-26T02:55:40Z,-- http://www.bloomberg.com/news/2013-11-26/ru...,after a forecast that the global surplus of th...
448391,-- SNB’s Jordan Sees No Reason to Remove Cap o...,-- B y C a t h e r i n e B o s l e y,-- 2013-11-26T00:01:00Z,-- http://www.bloomberg.com/news/2013-11-26/sn...,franc remains appropriate given the currency’s...
448392,-- U.K. Lawmakers Seek ‘Sharp’ Change as Bank ...,-- B y R o b e r t H u t t o n,-- 2013-11-26T00:01:00Z,-- http://www.bloomberg.com/news/2013-11-26/u-...,inquiry into banking standards will make a new...
448393,-- UBS Offers to Repurchase Some Shares of Pue...,-- B y L a u r a M a r c i n e k,-- 2013-11-26T00:00:01Z,-- http://www.bloomberg.com/news/2013-11-26/ub...,shares of its Puerto Rico bond funds after t...


In [4]:
text = df1.Artical[0]

In [5]:
import spacy 
import en_core_web_lg
nlp_spacy = en_core_web_lg.load()

In [6]:
text

"                                   Inco Ltd., the Canadian nickel producer being bought by  Brazil 's Cia. Vale do Rio, said third-quarter profit soared 11-fold, boosted by surging metal prices and fees paid by Falconbridge Ltd. after a failed takeover.    Net income jumped to $701 million, or $3.08 a share, from $64 million, or 29 cents, a year earlier, Toronto-based Inco said today in a statement. Results included $109 million in net fees from the failed deals with Falconbridge and Phelps Dodge Corp. Sales jumped to $2.32 billion from $1.08 billion.    Inco sold nickel at double the price last year on average, and output jumped 13 percent. Demand for the metal, used in stainless steel, surged as global economic growth fueled demand, especially in  China . Mines have failed to keep pace, prompting a buying spree by producers seeking to bolster ore deposits. Vale outbid Phelps Dodge and Teck Cominco Ltd. with its $17.3 billion bid.    ``Record quarterly earnings reflect the unpreceden

In [7]:
doc = nlp_spacy(text)

for ent in filter(lambda e: e.label_ == 'ORG', doc.ents):
       
       #current = ent.root
       #print(list(current.subtree))
   print("{} -> {} -> {}".format(ent.text, ent.root,ent.label_))

Cia -> Cia -> ORG
Vale -> Vale -> ORG
Falconbridge Ltd. -> Ltd. -> ORG
Inco -> Inco -> ORG
Falconbridge -> Falconbridge -> ORG
Phelps Dodge Corp. -> Corp. -> ORG
Vale -> Vale -> ORG
Phelps Dodge -> Dodge -> ORG
Teck Cominco Ltd. -> Ltd. -> ORG
the Toronto Stock Exchange -> Exchange -> ORG
Vale -> Vale -> ORG
Falconbridge -> Falconbridge -> ORG
Phelps Dodge -> Dodge -> ORG
LionOre Mining International Ltd. -> Ltd. -> ORG
Voisey's Bay in -> in -> ORG
OAO GMK Norilsk Nickel -> Nickel -> ORG
the London Metal Exchange -> Exchange -> ORG
the World Bureau of Metal Statistics -> Bureau -> ORG
LME Nickel    Nickel -> Nickel -> ORG
LME -> LME -> ORG


In [8]:
def dep_subtree(token, dep):
    #deps =[child.dep_ for child in token.children]
    child = next(filter(lambda c: c.dep_ == dep, token.children), None)
    
    if child != None:
        return " ".join([c.text for c in child.subtree])
    else:
        return ""

In [9]:
def extract_events_spacy(line):
    #ine=p.sub('', line)
    events = []
    doc = nlp_spacy(line)
    for ent in filter(lambda e: e.label_=='ORG',doc.ents):
        #print(ent)
        try:
            start = ent.text
        except:
        # could not parse the dates, hence ignore it
            continue
        current = ent.root
        while current.dep_ != "ROOT":
            current = current.head
        desc = " ".join(filter(None,[
                                    dep_subtree(current,"nsubj"),
                                    dep_subtree(current,"nsubjpass"),
                                    dep_subtree(current,"auxpass"),
                                    dep_subtree(current,"amod"),
                                    dep_subtree(current,"det"),
                                    current.text, 
                                    dep_subtree(current,"acl"),
                                    dep_subtree(current,"dobj"),
                                    dep_subtree(current,"attr"),
                                    dep_subtree(current,"advmod")
                                     ]))
        events = events + [(ent.text,desc)]
        
    return events

In [10]:
extract_events_spacy(text)


[('Cia',
  "                                    Inco Ltd. , the Canadian nickel producer being bought by   Brazil 's Cia . Vale do Rio , said"),
 ('Vale',
  "                                    Inco Ltd. , the Canadian nickel producer being bought by   Brazil 's Cia . Vale do Rio , said"),
 ('Falconbridge Ltd.',
  "                                    Inco Ltd. , the Canadian nickel producer being bought by   Brazil 's Cia . Vale do Rio , said"),
 ('Inco', 'Toronto - based Inco said'),
 ('Falconbridge', 'Results included $ 109 million in net fees'),
 ('Phelps Dodge Corp.', 'Results included $ 109 million in net fees'),
 ('Vale', 'Vale outbid Phelps Dodge and Teck Cominco Ltd.'),
 ('Phelps Dodge', 'Vale outbid Phelps Dodge and Teck Cominco Ltd.'),
 ('Teck Cominco Ltd.', 'Vale outbid Phelps Dodge and Teck Cominco Ltd.'),
 ('the Toronto Stock Exchange', 'Shares of Inco gained 27 cents'),
 ('Vale',
  "Rio de Janeiro - based Vale , the world 's largest iron - ore producer , offered C$ 86 a s

In [11]:
#nlp_spacy.add_pipe("merge_entities")
#lp_spacy.add_pipe("merge_noun_chunks")

TEXT =["Net income jumped to $701 million, or $3.08 a share, from $64 million, or 29 cents, a year earlier, Toronto-based Inco said today in a statement."]

for doc in nlp_spacy.pipe(TEXT):
    for token in doc:
        if token.ent_type_ == "MONEY":
            # We have an attribute and direct object, so check for subject
            if token.dep_ in ("attr", "dobj"):
                subj = [w for w in token.head.lefts if w.dep_ == "nsubj"]
                if subj:
                    print(subj[0], "-->", token)
            # We have a prepositional object with a preposition
            elif token.dep_ == "pobj" and token.head.dep_ == "prep":
                print(token.head.head, "-->", token)

jumped --> million
3.08 --> million


In [12]:
import stanza

In [13]:
nlp = stanza.Pipeline(lang='en', processors='tokenize,ner')
doc_stanza = nlp(text)
#print(*[f'entity: {ent.text}\ttype: {ent.type}' for sent in doc_stanza.sentences for ent in sent.ents], sep='\n')

2022-10-13 19:27:03 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2022-10-13 19:27:05 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| ner       | ontonotes |

2022-10-13 19:27:05 INFO: Use device: cpu
2022-10-13 19:27:05 INFO: Loading: tokenize
2022-10-13 19:27:05 INFO: Loading: ner
2022-10-13 19:27:05 INFO: Done loading processors!


In [14]:
for ents in filter(lambda e: e.type=='ORG',doc_stanza.ents):
    print(ents.text,"->", ents.type)

Inco Ltd. -> ORG
Cia -> ORG
Vale do Rio -> ORG
Falconbridge Ltd. -> ORG
Inco -> ORG
Falconbridge -> ORG
Phelps Dodge Corp -> ORG
Inco -> ORG
Vale -> ORG
Phelps Dodge -> ORG
Teck Cominco Ltd. -> ORG
Inco -> ORG
Inco -> ORG
the Toronto Stock Exchange -> ORG
Vale -> ORG
Inco -> ORG
Inco -> ORG
Falconbridge -> ORG
Phelps Dodge -> ORG
LionOre Mining International Ltd -> ORG
Voisey's Bay -> ORG
Inco -> ORG
Phelps Dodge -> ORG
Inco -> ORG
Vale -> ORG
Inco -> ORG
Inco -> ORG
Inco -> ORG
OAO GMK Norilsk Nickel -> ORG
the London Metal Exchange -> ORG
the World Bureau of Metal Statistics -> ORG
LME Nickel    Nickel -> ORG
LME -> ORG
Inco -> ORG
Inco -> ORG
Inco -> ORG
Inco -> ORG
CVRD -> ORG
Inco -> ORG
Goro -> ORG
Inco -> ORG
Goro -> ORG


In [15]:
events =[]
for sentences in doc.sents:
    for ent in filter(lambda e: e.label_=='ORG',sentences.ents):
        #print(ent)
        try:
            start = ent.text
        except:
        # could not parse the dates, hence ignore it
            continue
        current = ent.root
        while current.dep_ != "ROOT":
            current = current.head
        desc = " ".join(filter(None,[
                                    dep_subtree(current,"nsubj"),
                                    dep_subtree(current,"nsubjpass"),
                                    dep_subtree(current,"auxpass"),
                                    dep_subtree(current,"amod"),
                                    dep_subtree(current,"det"),
                                    current.text, 
                                    dep_subtree(current,"acl"),
                                    dep_subtree(current,"dobj"),
                                    dep_subtree(current,"attr"),
                                    dep_subtree(current,"advmod")
                                     ]))
        events = events + [(ent.text,desc)]
        
        
print(events)

[('Inco', 'Toronto - based Inco said')]


In [16]:
nlp_de = spacy.load('en_core_web_sm')

text_de = 'the guests have had been entertained by the music.'

doc_de = nlp_de(text_de)

predicates_elems = {}
for sents in doc.sents:
    print(sents)
    print("----------------------------------------")
    for token in sents:
        if token.dep_ == 'ROOT':
            predicates_elems[token.i] = token
            root_index = token.i

    for child in doc[root_index].children:
        if child.dep_ == 'aux' or child.dep_ == 'auxpass':
            predicates_elems[child.i] = child

items = list(predicates_elems.keys())
#items.sort()
#predicates= ' '.join([predicates_elems[ind] for ind in items])
predicates =[]
predicates.append(list(predicates_elems[ite] for ite in items))
#print("the predicates is:",predicates)
#print(len(items))

Net income jumped to $701 million, or $3.08 a share, from $64 million, or 29 cents, a year earlier, Toronto-based Inco said today in a statement.
----------------------------------------


In [17]:
for item in predicates:
    print(item)

[said]


In [15]:
# import os
# os.environ["CORENLP_HOME"] = r"C:\Users\Abdul\.stanfordnlp_resources\stanford-corenlp-4.1.0/"

In [13]:
import stanza
# Import client module
from stanza.server import CoreNLPClient

client = CoreNLPClient(timeout=150000000, be_quiet=True, annotators=['openie'], 
endpoint='http://localhost:65530')
client.start()
import time
time.sleep(10)

2022-10-16 00:23:06 INFO: Writing properties to tmp file: corenlp_server-d1eb8ed699d74af4.props
2022-10-16 00:23:06 INFO: Starting server with command: java -Xmx5G -cp C:\Users\Abdul\.stanfordnlp_resources\stanford-corenlp-4.1.0/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 65530 -timeout 150000000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-d1eb8ed699d74af4.props -annotators openie -preload -outputFormat serialized


In [ ]:
# netstat -ano | findstr:

#           // listening value
# taskkill /pid 11740 /f